## Echem

In [ ]:
# -*- coding: utf-8 -*-

import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import gridspec, ticker, transforms
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# 颜色设定
colors = tol_cset("vibrant")
if colors is not None:
    colors = list(colors)
else:
    # Fallback colors in case tol_cset returns None
    colors = ["#0077BB", "#33BBEE", "#009988", "#EE7733", "#CC3311", "#EE3377", "#BBBBBB"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

### Zn - Zn 迁移数

In [ ]:
path_file_ca = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\SuperP+Bacterial Cellulose\Results\Zn_Conductivity+TransferNumber\2M ZnSO4\YanGao"  # noqa: E501
    ).glob(r"**\*CA*.txt")
)
path_file_eis = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\SuperP+Bacterial Cellulose\Results\Zn_Conductivity+TransferNumber\2M ZnSO4\YanGao"  # noqa: E501
    ).glob(r"**\*PEIS*.txt")
)

ca = []
for path_file in path_file_ca:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # type: ignore  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    df.drop(columns=["Unnamed: 3"], inplace=True, errors="ignore")
    df['time'] = (df['time/s'] - df['time/s'].iloc[0]).dt.total_seconds()
    ca.append(df)

eis = []
for path_file in path_file_eis:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # type: ignore  # noqa: E501
    # 转换数据格式
    df[[r"freq/Hz", r"Re(Z)/Ohm", r"-Im(Z)/Ohm"]] = df[[r"freq/Hz", r"Re(Z)/Ohm", r"-Im(Z)/Ohm"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    eis.append(df)

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(1.0)

lss = ["-", "-", "--", "--"]
labels = [
    r"$\mathrm{GF-Before}$",
    r"$\mathrm{GF-After}$",
    r"$\mathrm{40 \ wt.\% \ Sp-Before}$",
    r"$\mathrm{40 \ wt.\% \ Sp-After}$",
]

for i, temp in enumerate(eis):
    ax.plot(
        temp['Re(Z)/Ohm'],
        temp['-Im(Z)/Ohm'],
        ls=lss[i],
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Re(Z)/Ohm}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 1000)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=200, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=100, offset=0))

ax.set_ylabel(r"$\mathrm{-Im(Z)/Ohm}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0, 1000)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=200, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=100, offset=0.0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.0, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

# axins = ax.inset_axes(  # noqa: ERA001
#     (0.55, 0.3, 0.35, 0.35),  # noqa: ERA001
#     zorder=1, # noqa: ERA001
#     transform=ax.transAxes, # noqa: ERA001
# ) # noqa: ERA001
# for i, temp in enumerate(eis):
#     axins.plot(
#         temp['Re(Z)/Ohm'], # noqa: ERA001
#         temp['-Im(Z)/Ohm'], # noqa: ERA001
#         ls=lss[i], # noqa: ERA001
#         lw=1.0, # noqa: ERA001
#         c=colors[i], # noqa: ERA001
#         label=labels[i], # noqa: ERA001
#         zorder=0, # noqa: ERA001
#     ) # noqa: ERA001
# axins.set_xlim(0, 10) # noqa: ERA001
# axins.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0)) # noqa: ERA001
# axins.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0)) # noqa: ERA001
# axins.set_ylim(0, 10) # noqa: ERA001
# axins.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0.0)) # noqa: ERA001
# axins.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0.0)) # noqa: ERA001
# axins.tick_params(axis="both", direction="out", labelsize=8) # noqa: ERA001

# ax.indicate_inset_zoom(axins, edgecolor="grey", alpha=0.5, lw=1.0, ls=r"--", zorder=2) # noqa: ERA001

# 图
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)
labels = [
    [r"$\mathrm{GF}$", None, None, None],
    [r"$\mathrm{40 \ wt.\% \ Sp}$", None, None, None],
    [r"$\mathrm{3^{rd}}$", None, None, None],
    [r"$\mathrm{4^{th}}$", None, None, None],
]
for i, data in enumerate(ca):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_min = temp["<I>/mA"].idxmin()
        # 断开最小值前后的曲线
        ax.plot(
            temp.loc[:idx_min, "time"]/60,
            temp.loc[:idx_min, "<I>/mA"]*1000,
            ls="-",
            lw=1.0,
            c=colors[i],
            label=labels[i][j],
            zorder=0,
        )


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Duration \ Time \ (minute)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 120)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=30, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=15, offset=0))

ax.set_ylabel(r"$\mathrm{Current \ (\mu A)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0, 250)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0.0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

ax.text(
    0.05,
    0.07,
    r"$\mathrm{2M \ ZnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

ax.text(
    0.35,
    0.6,
    r'$\mathrm{t_{diffusion}} = \frac{I_s (\Delta V - I_0 k_0)}{I_0 (\Delta V - I_s k_s)}$',
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=14,
    c="k",
)

ax.text(
    0.35,
    0.4,
    r'$\mathrm{t_{GF} : 0.0405}$',
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=11,
    c=colors[0],
)

ax.text(
    0.35,
    0.3,
    r"$\mathrm{40 \ wt.\% \ Sp : 0.0914}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=11,
    c=colors[1],
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_BC_1_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_BC_1_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

### BC@SuperP，EIS

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\SuperP+Bacterial Cellulose\Results\Zn_Conductivity+TransferNumber\2M ZnSO4\2024-ICMAB\Zn-Zn"  # noqa: E501
)
file = pd.read_csv(
    Path.joinpath(path_file, r"Results", r"EIS_Zn_Zn_Conductivity.csv"), sep=",", skiprows=1, header=0, comment="#"
)

In [ ]:
%matplotlib inline

plt.close("all")
fig = plt.figure(figsize=(9, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)

labels_a = [r"Glass Fiber", r"SP_0%", r"SP_10%", r"SP_20%", r"SP_40%", r"SP_60%", r"SP_80%", r"SP_100%"]

for i in range(len(labels_a)):
    ax.scatter(file.iloc[:, 2 * i], file.iloc[:, 2 * i + 1], c=colors[i])
    ax.plot(file.iloc[:, 2 * i], file.iloc[:, 2 * i + 1], c=colors[i], ls="-", lw=1.0, label=labels_a[i])

ax.set_xlim(0, 2000)
ax.set_xlabel(r"Re(Z) (ohm)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=500))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=250))
ax.set_ylim(0, 2000)
ax.set_ylabel(r"-Im(Z) (ohm)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=500))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=250))
ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.65, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
axins = ax.inset_axes(
    (0.10, 0.4, 0.55, 0.55),
    zorder=3,
)
for i in range(len(labels_a) - 4):
    j = i + 4
    axins.scatter(file.iloc[:, 2 * j], file.iloc[:, 2 * j + 1], c=colors[j], s=15)
    axins.plot(file.iloc[:, 2 * j], file.iloc[:, 2 * j + 1], c=colors[j], ls="-", lw=1.0, label=labels_a[j])
    axins.set_xlim(0, 12.5)
    axins.xaxis.set_major_locator(ticker.MultipleLocator(base=2))
    axins.xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
    axins.set_ylim(0, 12.5)
    axins.yaxis.set_major_locator(ticker.MultipleLocator(base=3))
    axins.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5))
    axins.tick_params(axis="x", labelsize=9)
    axins.tick_params(axis="y", labelsize=9)
    axins.legend(loc="upper left", bbox_to_anchor=(0.01, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=7)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_axes((0.25, 0, 1, 1), zorder=0)

data = pd.concat([file.iloc[:, 6:8], file.iloc[:, 10:12], file.iloc[:, 16:20]], axis=1, ignore_index=True)
labels_b = [r"SP_20%", r"SP_60%", r"SP_20%_air", r"SP_60%_air"]

for i in range(len(labels_b)):
    ax.scatter(data.iloc[:, 2 * i], data.iloc[:, 2 * i + 1], c=colors[i])
    ax.plot(data.iloc[:, 2 * i], data.iloc[:, 2 * i + 1], c=colors[i], ls="-", lw=1.0, label=labels_b[i])

ax.set_xlim(0, 350)
ax.set_xlabel(r"Re(Z) (ohm)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25))
ax.set_ylim(0, 350)
ax.set_ylabel(r"-Im(Z) (ohm)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=50))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=25))
ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=9)

ax.legend(loc="upper left", bbox_to_anchor=(0.63, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

axins = ax.inset_axes(
    (0.10, 0.4, 0.55, 0.55),
    zorder=3,
)
for i in range(len(labels_b) - 2):
    j = 4 * i + 2
    axins.scatter(data.iloc[:, j], data.iloc[:, j + 1], c=colors[2 * i + 1], s=15)
    axins.plot(data.iloc[:, j], data.iloc[:, j + 1], c=colors[2 * i + 1], ls="-", lw=1.0, label=labels_b[2 * i + 1])
    axins.set_xlim(0, 12.5)
    axins.xaxis.set_major_locator(ticker.MultipleLocator(base=2))
    axins.xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
    axins.set_ylim(0, 3)
    axins.yaxis.set_major_locator(ticker.MultipleLocator(base=1))
    axins.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
    axins.tick_params(axis="x", labelsize=9)
    axins.tick_params(axis="y", labelsize=9)
    axins.legend(loc="upper left", bbox_to_anchor=(0.01, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=7)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_axes((0.45, 0, 1, 1), zorder=0)

ax.scatter(file.iloc[:, 8], file.iloc[:, 9], c=colors[4], alpha=0.5)
ax.plot(file.iloc[:, 8], file.iloc[:, 9], c=colors[4], ls="-", lw=1.0, label=labels_a[4], alpha=0.5)
ax.scatter(file.iloc[:, -2], file.iloc[:, -1], c=colors[4])
ax.plot(file.iloc[:, -2], file.iloc[:, -1], c=colors[4], ls="-", lw=1.0, label=r"SP_40%_2")


ax.set_xlim(2, 7.5)
ax.set_xlabel(r"Re(Z) (ohm)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.set_ylim(0, 1.0)
ax.set_ylabel(r"-Im(Z) (ohm)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.02, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

plt.savefig(
    Path.joinpath(path_out, r"Cellulose_Echem_EIS_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"Cellulose_Echem_EIS_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### BC@SuperP@aMnO2 = 524

#### GCD, C/3

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2@SuperP+Bacterial Cellulose\40%SuperP_Bacterial Cellulose_32αMnO2\1M ZnSO4 + 0.2M MnSO4\GCD\Results"  # noqa: E501, RUF001
)
file = pd.read_excel(
    Path.joinpath(
        path_file,
        r"LC-03C-Zn-1M ZnSO4 + 02M MnSO4-BCaMnO2-1.7mg-5232_20250117174520_DefaultGroup_00_1(M340A320302052).xlsx",
    ),  # noqa: E501
    sheet_name=2,
    skiprows=0,
    header=0,
    comment="#",
)
file = file[file["Cycle"].isin([1, 5, 10])]

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

labels = [r"$\mathrm{1^{st}}$", r"$\mathrm{5^{th}}$", r"$\mathrm{10^{th}}$", r"$\mathrm{50^{th}}$"]
for i, idx in enumerate(file["Cycle"].unique()):
    temp = file[file["Cycle"] == idx]
    # 找到电压最小值的索引
    idx_min = temp[temp["Current/uA"] < 0]["Voltage/V"].idxmin()
    # 断开最小值前后的曲线
    ax.plot(
        temp.loc[: idx_min - 1, "SpeCap/mAh/g"],
        temp.loc[: idx_min - 1, "Voltage/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )  # noqa: E501
    ax.plot(
        temp.loc[idx_min + 1 :, "SpeCap/mAh/g"],
        temp.loc[idx_min + 1 :, "Voltage/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=None,
        zorder=0,
    )  # noqa: E501

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO_2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-1, 40)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=8, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=4, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.1,
    r"$\mathrm{C/3, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)  # noqa: E501

ax.legend(loc="upper left", bbox_to_anchor=(0.7, 0.75), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_BC_524_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_BC_524_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### CE, 3C

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2@SuperP+Bacterial Cellulose\40%SuperP_Bacterial Cellulose_32αMnO2\1M ZnSO4 + 0.2M MnSO4\GCD\Results"  # noqa: E501, RUF001
)
file = pd.read_excel(
    Path.joinpath(
        path_file,
        r"LC-03C-Zn-1M ZnSO4 + 02M MnSO4-BCaMnO2-1.7mg-5232_20250117174520_DefaultGroup_00_1(M340A320302052)(2).xlsx",
    ),  # noqa: E501
    sheet_name=2,
    skiprows=0,
    header=0,
    comment="#",
)
file.drop(index=file.index[1], inplace=True)
file.reset_index(drop=True, inplace=True)
file["CE"] = file["SpeCapD/mAh/g"] / file["SpeCapC/mAh/g"]

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(file.iloc[0:500, 1], c=colors[0], ls="-", lw=1.0, label=r"Charge", marker="o", ms=5)
ax.plot(file.iloc[0:500, 2], c=colors[1], ls="-", lw=1.0, label=r"Discharge", marker="o", ms=5)
ax.set_ylim(0, 60)
ax.set_ylabel(r"$\mathrm{Capacity \ (mAh \ g^{-1}_{MnO_2})}$", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=12))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=6))
ax.set_xlim(0, 150)
ax.set_xlabel(r"Cycle Number (N)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=30))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=15))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 0.7), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}  # type: ignore
ax.annotate(
    r" ", xy=(0.2, 0.35), xycoords="axes fraction", xytext=(0, 0.25), textcoords="axes fraction", arrowprops=arrowprops
)
ax.text(
    0.05,
    0.07,
    r"$\mathrm{C/3, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)  # noqa: E501

# 插入图
ax2 = ax.twinx()
ax2.set_position((0.0, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.plot(file.iloc[0:500, 4] * 100, c=colors[2], ls="-", lw=1.0, label=r"Coulomb Efficiency", marker="o", ms=5)

ax2.set_ylim(0, 105)
ax2.set_ylabel(r"Coulomb Efficiency (%)", fontsize=11)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=20))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=10))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9)
ax2.legend(loc="upper left", bbox_to_anchor=(0.5, 0.56), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.6,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
    "ls": r"-",
}  # noqa: E501
ax2.annotate(  # type: ignore
    r" ",
    xy=(0.8, 0.9),
    xycoords="axes fraction",
    xytext=(1.0, 0.8),
    textcoords="axes fraction",
    arrowprops=arrowprops,  # noqa: E501
)

plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_Pvdf_524_2_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_Pvdf_524_2_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### BC@SuperP@aMnO2, 30mg-60mg

#### CE

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2@SuperP+Bacterial Cellulose\30mg_αMnO2_60mg_SuperP@Bacterial Cellulose\GCD\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
)
file = pd.read_excel(
    Path.joinpath(
        path_file,
        r"LC-03C-Zn-1M+02M-celluloseMnO2(60-30)_15.81_2.2mg_20230616124830_DefaultGroup_00_8(M340A320302052)(2).xlsx",
    ),  # noqa: E501
    sheet_name=2,
    skiprows=0,
    header=0,
    comment="#",
)
file.drop(index=file.index[1], inplace=True)
file.reset_index(drop=True, inplace=True)
file["CE"] = file["SpeCapD/mAh/g"] / file["SpeCapC/mAh/g"]

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(file.iloc[0:59, 1], c=colors[0], ls="-", lw=1.0, label=r"Charge", marker="o", ms=5)
ax.plot(file.iloc[0:59, 2], c=colors[1], ls="-", lw=1.0, label=r"Discharge", marker="o", ms=5)
ax.set_ylim(100, 600)
ax.set_ylabel(r"$\mathrm{Capacity \ (mAh \ g^{-1}_{MnO_2})}$", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=100))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=50))
ax.set_xlim(0, 60)
ax.set_xlabel(r"Cycle Number (N)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5))
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 0.35), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}  # type: ignore
ax.annotate(
    r" ", xy=(0.2, 0.35), xycoords="axes fraction", xytext=(0, 0.25), textcoords="axes fraction", arrowprops=arrowprops
)

# 插入图
ax2 = ax.twinx()
ax2.set_position((0.0, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.plot(file.iloc[0:59, 4] * 100, c=colors[2], ls="-", lw=1.0, label=r"Coulomb Efficiency", marker="o", ms=5)

ax2.set_ylim(0, 105)
ax2.set_ylabel(r"Coulomb Efficiency (%)", fontsize=11)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=20))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=10))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9)
ax2.legend(loc="upper left", bbox_to_anchor=(0.5, 0.2), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.6,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
    "ls": r"-",
}  # noqa: E501
ax2.annotate(  # type: ignore
    r" ", xy=(0.8, 0.9), xycoords="axes fraction", xytext=(1.0, 0.8), textcoords="axes fraction", arrowprops=arrowprops
)

plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_CE_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_CE_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### GCD

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2@SuperP+Bacterial Cellulose\30mg_αMnO2_60mg_SuperP@Bacterial Cellulose\GCD\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
)
file = pd.read_excel(
    Path.joinpath(
        path_file,
        r"LC-03C-Zn-1M+02M-celluloseMnO2(60-30)_15.81_2.2mg_20230616124830_DefaultGroup_00_8(M340A320302052).xlsx",
    ),  # noqa: E501
    sheet_name=2,
    skiprows=0,
    header=0,
    comment="#",
)
file = file[file["Cycle"].isin([1, 5, 10, 50])]

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

labels = [r"$\mathrm{1^{st}}$", r"$\mathrm{5^{th}}$", r"$\mathrm{10^{th}}$", r"$\mathrm{50^{th}}$"]
for i, idx in enumerate(file["Cycle"].unique()):
    temp = file[file["Cycle"] == idx]
    # 找到电压最小值的索引
    idx_min = temp[temp["Current/uA"] < 0]["Voltage/V"].idxmin()
    # 断开最小值前后的曲线
    ax.plot(
        temp.loc[: idx_min - 1, "SpeCap/mAh/g"],
        temp.loc[: idx_min - 1, "Voltage/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )  # noqa: E501
    ax.plot(
        temp.loc[idx_min + 1 :, "SpeCap/mAh/g"],
        temp.loc[idx_min + 1 :, "Voltage/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=None,
        zorder=0,
    )  # noqa: E501

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO_2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-1, 450)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=90, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=60, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.text(
    0.05,
    0.1,
    r"$\mathrm{C/3, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)  # noqa: E501

ax.legend(loc="upper left", bbox_to_anchor=(0.7, 0.8), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_GCD_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_GCD_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### BC@SuperP@aMnO2, alpha MnO2 + PVDF + Sp = 524

#### GCD, C/3

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2\50PVDF20SP40αMnO2\GCD\1M ZnSO4 +0.2M MnSO4\Results"  # noqa: E501, RUF001
)
file = pd.read_excel(
    Path.joinpath(path_file, r"LC-1Ag-Zn-1M ZnSO4 + 02M MnSO4-aMnO2-5.3mg-524_20250121095932_DefaultGroup_00_1(M340A320302052).xlsx"),  # noqa: E501
    sheet_name=2,
    skiprows=0,
    header=0,
    comment="#",
)
file = file[file["Cycle"].isin([1, 5, 10])]

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

labels = [r"$\mathrm{1^{st}}$", r"$\mathrm{5^{th}}$", r"$\mathrm{10^{th}}$", r"$\mathrm{50^{th}}$"]
for i, idx in enumerate(file["Cycle"].unique()):
    temp = file[file["Cycle"] == idx]
    # 找到电压最小值的索引
    idx_min = temp[temp["Current/uA"] < 0]["Voltage/V"].idxmin()
    # 断开最小值前后的曲线
    ax.plot(
        temp.loc[: idx_min - 1, "SpeCap/mAh/g"],
        temp.loc[: idx_min - 1, "Voltage/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )  # noqa: E501
    ax.plot(
        temp.loc[idx_min + 1 :, "SpeCap/mAh/g"],
        temp.loc[idx_min + 1 :, "Voltage/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=None,
        zorder=0,
    )  # noqa: E501

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO_2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-1, 300)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=60, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=30, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.1,
    r"$\mathrm{3C, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)  # noqa: E501

ax.legend(loc="upper left", bbox_to_anchor=(0.7, 0.75), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)

plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_Pvdf_524_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_Pvdf_524_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### CE, 3C

In [ ]:
# 读取数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperTres\Echem\Bacterial Cellulose\αMnO2@SuperP+Bacterial Cellulose\40%SuperP_Bacterial Cellulose_8αMnO2\GCD\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
)
file = pd.read_excel(
    Path.joinpath(
        path_file,
        r"LC-Cellulose-01C-Zn-1M_ZnSO4+02M_MnSO4-alphaMnO2-1-8-5.1mg_20240812195008_DefaultGroup_01_8(2).xlsx",
    ),  # noqa: E501
    sheet_name=2,
    skiprows=0,
    header=0,
    comment="#",
)
file.drop(index=file.index[1], inplace=True)
file.reset_index(drop=True, inplace=True)
file["CE"] = file["SpeCapD/mAh/g"] / file["SpeCapC/mAh/g"]

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.plot(file.iloc[0:500, 1], c=colors[0], ls="-", lw=1.0, label=r"Charge", marker="o", ms=5)
ax.plot(file.iloc[0:500, 2], c=colors[1], ls="-", lw=1.0, label=r"Discharge", marker="o", ms=5)
ax.set_ylim(0, 160)
ax.set_ylabel(r"$\mathrm{Capacity \ (mAh \ g^{-1}_{MnO_2})}$", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=40))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=20))
ax.set_xlim(0, 50)
ax.set_xlabel(r"Cycle Number (N)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
arrowprops = {"arrowstyle": "<-", "color": colors[0], "connectionstyle": "angle,angleA=0,angleB=90,rad=10"}  # type: ignore
ax.annotate(
    r" ", xy=(0.2, 0.25), xycoords="axes fraction", xytext=(0, 0.15), textcoords="axes fraction", arrowprops=arrowprops
)
ax.text(
    0.05,
    0.1,
    r"$\mathrm{C/10, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)  # noqa: E501

# 插入图
ax2 = ax.twinx()
ax2.set_position((0.0, 0.0, 1.0, 1.0))
ax2.set_box_aspect(0.8)
ax2.plot(file.iloc[0:500, 4] * 100, c=colors[2], ls="-", lw=1.0, label=r"Coulomb Efficiency", marker="o", ms=5)

ax2.set_ylim(0, 105)
ax2.set_ylabel(r"Coulomb Efficiency (%)", fontsize=11)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=20))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=10))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9)
ax2.legend(loc="upper left", bbox_to_anchor=(0.5, 0.46), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
arrowprops = {
    "arrowstyle": "<-",
    "color": colors[1],
    "alpha": 0.6,
    "connectionstyle": "angle,angleA=0,angleB=90,rad=10",
    "ls": r"-",
}  # noqa: E501
ax2.annotate(  # type: ignore
    r" ", xy=(0.8, 0.7), xycoords="axes fraction", xytext=(1.0, 0.8), textcoords="axes fraction", arrowprops=arrowprops
)

plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_Pvdf_524_2_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2@Sp@Cellulose_Echem_Pvdf_524_2_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### CV

#### 读取并且合并不同扫速的 CV 数据

In [ ]:
# 读取数据
path_file_list = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2\50PVDF20SP40αMnO2\CV\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
    ).glob("*.txt")
)

echem = {}
for path_file in path_file_list:
    line_skip = scan_rate = None
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
            elif line.startswith("dE/dt"):
                scan_rate = float(line.split()[1])
            if line_skip and scan_rate is not None:
                break
    df = pd.read_csv(
        path_file,
        sep=r"\t",
        comment="#",
        skiprows=line_skip if line_skip is not None else 0,
        engine="python",
        encoding="latin_1",
        decimal=".",
        header=None,
        index_col=None,
    ).dropna(axis=1, how="all")  # noqa: E501
    df.columns = ["cycle number", "time/s", "Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]  # noqa: E501
    df.drop(columns=["time/s", "Ece/V", "Capacity/mA.h"], inplace=True, errors="ignore")  # 删除不需要的列
    df[["Ewe/V", "<I>/mA"]] = df[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    df = df[df["cycle number"].isin([1, 2])]  # 保留第1+1圈
    idx_min = df[df["cycle number"] == 1]["Ewe/V"].idxmax()  # noqa: PLR2004
    idx_max = df[df["cycle number"] == 2]["Ewe/V"].idxmax()  # noqa: PLR2004
    df = df.loc[idx_min:idx_max].reset_index(drop=True)  # noqa: PLR2004
    echem[f"{scan_rate} mV/s"] = df

In [ ]:
lengths = {k: v.shape for k, v in echem.items()}
for k, v in lengths.items():
    print(f"{k}: {v}")

# 获取所有 df 的 index 交集
common_index = None
for df in echem.values():
    common_index = df.index if common_index is None else common_index.intersection(df.index)

for key in echem:  # noqa: PLC0206
    echem[key] = echem[key].reindex(common_index)

current = pd.DataFrame(np.array(list(echem.values()))[:, :, 2]).T  # units: mA
voltage = pd.DataFrame(np.array(list(echem.values()))[:, :, 1]).T  # units: V

current.columns = [f"mA{i + 1}" for i in range(current.shape[1])]
voltage.columns = list(echem.keys())

# 按 interleave 交错顺序重新排列
interleaved_columns = [item for pair in zip(voltage.columns.to_list(), current.columns.to_list()) for item in pair]

results = pd.concat([voltage, current], axis=1, ignore_index=False)
results = results[interleaved_columns]  # 按照交错顺序重新排列
results.to_csv(path_out.joinpath("CV_results.csv"), index=False, header=True)

In [ ]:
%matplotlib widget
from scipy.signal import find_peaks, savgol_filter

num_peaks = 3

# 2. 遍历每一条电流曲线
for index, col in enumerate(current.columns):
    i_data = current[col].values

    # 1. 数据预处理（去除 NaN 和异常值）
    i_data = pd.Series(i_data).interpolate().fillna(0).values
    # 3. 平滑处理（使用 Savitzky-Golay 滤波器）
    smoothed = savgol_filter(i_data, window_length=11, polyorder=3)

    # 4. 正峰检测（氧化峰）
    peaks_pos, _ = find_peaks(smoothed, prominence=0.001)
    peak_vals_pos = smoothed[peaks_pos]
    top_indices_pos = peaks_pos[np.argsort(peak_vals_pos)[-num_peaks:][::-1]]

    # 5. 负峰检测（还原峰）
    peaks_neg, _ = find_peaks(-smoothed, prominence=0.001)
    peak_vals_neg = smoothed[peaks_neg]
    top_indices_neg = peaks_neg[np.argsort(-peak_vals_neg)[-num_peaks:][::-1]]

    # 6. 绘图
    plt.figure(figsize=(10, 4))
    plt.plot(i_data, label="Raw data", alpha=0.4)
    plt.plot(smoothed, label="Smoothed", linewidth=2)

    # 正峰标注
    plt.plot(top_indices_pos, smoothed[top_indices_pos], "ro", label="Oxidation Peaks")
    for i in top_indices_pos:
        plt.annotate(
            f"{smoothed[i]:.2f}",
            xy=(i, smoothed[i]),
            xytext=(i, smoothed[i] + 0.02),
            arrowprops={"arrowstyle": "->", "color": "red"},
            fontsize=8,
        )

    # 负峰标注
    plt.plot(top_indices_neg, smoothed[top_indices_neg], "bo", label="Reduction Peaks")
    for i in top_indices_neg:
        plt.annotate(
            f"{smoothed[i]:.2f}",
            xy=(i, smoothed[i]),
            xytext=(i, smoothed[i] - 0.02),
            arrowprops={"arrowstyle": "->", "color": "blue"},
            fontsize=8,
        )

    # 图形设置
    plt.title(f"Peak Detection for {col}")
    plt.xlabel("Index (a.u.)")
    plt.ylabel("Current (mA)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#### CV 画图

In [ ]:
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2\50PVDF20SP40αMnO2\CV\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
)  # noqa: E501, RUF001
data = pd.read_excel(
    path_file.joinpath("CV_50PVDF20SP40αMnO2.xlsx"),
    engine="openpyxl",
    sheet_name=0,
).dropna(axis=1, how="all")  # noqa: E501, RUF001
data = data.iloc[:, 26:]

In [ ]:
%matplotlib inline

plt.close("all")
scan_rate = [r"0.1 mV/s", r"0.3 mV/s", r"0.5 mV/s", r"0.7 mV/s", r"1.0 mV/s"]

fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(data.shape[1] // 4):
    ax.plot(
        data.iloc[:, i * 4],
        data.iloc[:, i * 4 + 1],
        color=colors[i],
        label=scan_rate[i],
        linewidth=1.0,
    )

# 设置刻度线等格式
ax.set_ylabel(r"$\mathrm{Current \ (mA)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.8, 1.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=-0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.4, offset=-0.2))

ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.8, 2.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(-0.08, 1.0),
    ncols=2,
    handlelength=0.0,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.3,
)

plt.savefig(
    Path.joinpath(path_out, r"MnO2_CV_all_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2_CV_all_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline
ylims_params = [
    (-0.2, 0.3, 0.1, 0.05, 0, 0),
    (-0.3, 0.7, 0.2, 0.1, 0.1, 0.1),
    (-0.5, 1.0, 0.3, 0.15, 0.1, 0.1),
    (-0.6, 1.2, 0.6, 0.3, 0, 0),
    (-0.8, 1.6, 0.6, 0.3, -0.2, -0.2),
]

for i in range(data.shape[1] // 4):
    # 计算贡献
    area_all = np.trapz(data.iloc[:, i * 4 + 1], data.iloc[:, i * 4])
    area_capacitance = np.trapz(data.iloc[:, i * 4 + 2], data.iloc[:, i * 4])
    ratio = 100 * area_capacitance / area_all if area_all != 0 else 0

    fig = plt.figure(figsize=(3.3, 2.5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

    # 图
    subfig = fig.add_subfigure(gs[0, 0], zorder=0)
    ax = subfig.add_subplot()
    ax.set_position((0, 0, 1.0, 1.0))
    ax.set_box_aspect(0.8)

    ax.plot(
        data.iloc[:, i * 4],
        data.iloc[:, i * 4 + 1],
        color=colors[0],
        label="Data",
        linewidth=1.0,
    )
    ax.plot(
        data.iloc[:, i * 4],
        data.iloc[:, i * 4 + 2],
        color=colors[1],
        label="Capacitance",
        linewidth=1.0,
    )

    # 设置刻度线等格式
    ax.set_ylabel(r"$\mathrm{Current \ (mA)}$", fontsize=11, labelpad=1.0)
    ax.set_ylim(ylims_params[i][0], ylims_params[i][1])
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=ylims_params[i][2], offset=ylims_params[i][4]))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=ylims_params[i][3], offset=ylims_params[i][5]))

    ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
    ax.set_xlim(0.8, 2.0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0))

    ax.tick_params(axis="both", direction="out", labelsize=9)
    ax.legend(
        loc="upper left",
        bbox_to_anchor=(-0.01, 1.0),
        ncols=1,
        handlelength=0.8,
        frameon=False,
        labelcolor="linecolor",
        fontsize=9,
    )
    ax.text(
        0.05, 0.55, f"Ratio {ratio:.1f} %", transform=ax.transAxes, fontsize=9, color=colors[1], ha="left", va="top"
    )

    plt.savefig(
        Path.joinpath(path_out, f"MnO2_CV_{i}_300.tif"),
        transparent=False,
        pad_inches=0.05,
        bbox_inches="tight",
        dpi=300,
        pil_kwargs={"compression": "tiff_lzw"},
    )
    plt.savefig(
        Path.joinpath(path_out, f"MnO2_CV_{i}_600.tif"),
        transparent=False,
        pad_inches=0.05,
        bbox_inches="tight",
        dpi=600,
        pil_kwargs={"compression": "tiff_lzw"},
    )
plt.gcf().set_facecolor("white")
plt.show()